In [40]:
import cv2
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import csv
import time
import pickle

In [41]:
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

"""
Creates no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels


GPU is active
Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000


In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/vgg_resize/'
outpath = 'gen_data/vgg_outputs_app50k_'

random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state, 
                                        max_iter = 10))
classifier_sgd = OneVsRestClassifier(linear_model.SGDClassifier(max_iter=10, tol=1e-3))
    
i = 1
file_list = sorted(os.listdir(inputpath))
train_list = file_list[:54000]
np_data = np.empty(shape=[0, 4096])
np_label = np.empty(shape=[0,9])

for file_name in train_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    
    #classifier_sgd.partial_fit(data, label, classes=2)
    np_data = np.append(np_data, data, axis = 0)
    np_label = np.append(np_label, label, axis = 0)
    
    #print(label, label.shape)
    #break
    i = i + 1
 
sio.savemat(outpath + 'data' + '.vgg', {'data':np_data}, do_compression=True)
sio.savemat(outpath + 'label' + '.vgg', {'label':np_label}, do_compression=True)

#classifier.fit(np_data, np_label)    
    

10.jpg.vgg.mat - 1
100009.jpg.vgg.mat - 2
100021.jpg.vgg.mat - 3
100032.jpg.vgg.mat - 4
100034.jpg.vgg.mat - 5
100036.jpg.vgg.mat - 6
100039.jpg.vgg.mat - 7
10004.jpg.vgg.mat - 8
100043.jpg.vgg.mat - 9
100048.jpg.vgg.mat - 10
100061.jpg.vgg.mat - 11
100073.jpg.vgg.mat - 12
100087.jpg.vgg.mat - 13
100088.jpg.vgg.mat - 14
10009.jpg.vgg.mat - 15
100093.jpg.vgg.mat - 16
100097.jpg.vgg.mat - 17
1001.jpg.vgg.mat - 18
100100.jpg.vgg.mat - 19
100101.jpg.vgg.mat - 20
100105.jpg.vgg.mat - 21
100113.jpg.vgg.mat - 22
100135.jpg.vgg.mat - 23
10014.jpg.vgg.mat - 24
100147.jpg.vgg.mat - 25
10016.jpg.vgg.mat - 26
100163.jpg.vgg.mat - 27
100168.jpg.vgg.mat - 28
100179.jpg.vgg.mat - 29
100180.jpg.vgg.mat - 30
100181.jpg.vgg.mat - 31
100184.jpg.vgg.mat - 32
100185.jpg.vgg.mat - 33
1002.jpg.vgg.mat - 34
100206.jpg.vgg.mat - 35
100209.jpg.vgg.mat - 36
10022.jpg.vgg.mat - 37
100223.jpg.vgg.mat - 38
10023.jpg.vgg.mat - 39
100233.jpg.vgg.mat - 40
100235.jpg.vgg.mat - 41
100237.jpg.vgg.mat - 42
100239.jpg.vgg.

102120.jpg.vgg.mat - 364
102127.jpg.vgg.mat - 365
102128.jpg.vgg.mat - 366
102130.jpg.vgg.mat - 367
102131.jpg.vgg.mat - 368
102134.jpg.vgg.mat - 369
102138.jpg.vgg.mat - 370
102140.jpg.vgg.mat - 371
102149.jpg.vgg.mat - 372
102151.jpg.vgg.mat - 373
102154.jpg.vgg.mat - 374
10216.jpg.vgg.mat - 375
102170.jpg.vgg.mat - 376
102171.jpg.vgg.mat - 377
102176.jpg.vgg.mat - 378
10218.jpg.vgg.mat - 379
102180.jpg.vgg.mat - 380
102187.jpg.vgg.mat - 381
102188.jpg.vgg.mat - 382
102193.jpg.vgg.mat - 383
102199.jpg.vgg.mat - 384
102202.jpg.vgg.mat - 385
102203.jpg.vgg.mat - 386
102206.jpg.vgg.mat - 387
102210.jpg.vgg.mat - 388
102211.jpg.vgg.mat - 389
102213.jpg.vgg.mat - 390
102224.jpg.vgg.mat - 391
102227.jpg.vgg.mat - 392
102229.jpg.vgg.mat - 393
10223.jpg.vgg.mat - 394
102237.jpg.vgg.mat - 395
102249.jpg.vgg.mat - 396
102251.jpg.vgg.mat - 397
102256.jpg.vgg.mat - 398
102260.jpg.vgg.mat - 399
102269.jpg.vgg.mat - 400
102282.jpg.vgg.mat - 401
102298.jpg.vgg.mat - 402
102302.jpg.vgg.mat - 403
102

10418.jpg.vgg.mat - 718
104189.jpg.vgg.mat - 719
104192.jpg.vgg.mat - 720
104196.jpg.vgg.mat - 721
104205.jpg.vgg.mat - 722
104206.jpg.vgg.mat - 723
104208.jpg.vgg.mat - 724
104216.jpg.vgg.mat - 725
10422.jpg.vgg.mat - 726
104223.jpg.vgg.mat - 727
104228.jpg.vgg.mat - 728
104242.jpg.vgg.mat - 729
104254.jpg.vgg.mat - 730
104255.jpg.vgg.mat - 731
104262.jpg.vgg.mat - 732
104275.jpg.vgg.mat - 733
104277.jpg.vgg.mat - 734
104279.jpg.vgg.mat - 735
104285.jpg.vgg.mat - 736
104288.jpg.vgg.mat - 737
104291.jpg.vgg.mat - 738
104297.jpg.vgg.mat - 739
104313.jpg.vgg.mat - 740
104327.jpg.vgg.mat - 741
104331.jpg.vgg.mat - 742
104333.jpg.vgg.mat - 743
104336.jpg.vgg.mat - 744
104340.jpg.vgg.mat - 745
104348.jpg.vgg.mat - 746
104350.jpg.vgg.mat - 747
104354.jpg.vgg.mat - 748
104356.jpg.vgg.mat - 749
104357.jpg.vgg.mat - 750
104365.jpg.vgg.mat - 751
104366.jpg.vgg.mat - 752
104378.jpg.vgg.mat - 753
104387.jpg.vgg.mat - 754
104394.jpg.vgg.mat - 755
104398.jpg.vgg.mat - 756
104399.jpg.vgg.mat - 757
10

106082.jpg.vgg.mat - 1071
106094.jpg.vgg.mat - 1072
106107.jpg.vgg.mat - 1073
106110.jpg.vgg.mat - 1074
106117.jpg.vgg.mat - 1075
106119.jpg.vgg.mat - 1076
106120.jpg.vgg.mat - 1077
106125.jpg.vgg.mat - 1078
106126.jpg.vgg.mat - 1079
106134.jpg.vgg.mat - 1080
106135.jpg.vgg.mat - 1081
106136.jpg.vgg.mat - 1082
106140.jpg.vgg.mat - 1083
106145.jpg.vgg.mat - 1084
106146.jpg.vgg.mat - 1085
106147.jpg.vgg.mat - 1086
106148.jpg.vgg.mat - 1087
106153.jpg.vgg.mat - 1088
106164.jpg.vgg.mat - 1089
106170.jpg.vgg.mat - 1090
106171.jpg.vgg.mat - 1091
106173.jpg.vgg.mat - 1092
106185.jpg.vgg.mat - 1093
106186.jpg.vgg.mat - 1094
10620.jpg.vgg.mat - 1095
106201.jpg.vgg.mat - 1096
106211.jpg.vgg.mat - 1097
106213.jpg.vgg.mat - 1098
106214.jpg.vgg.mat - 1099
106218.jpg.vgg.mat - 1100
106227.jpg.vgg.mat - 1101
106232.jpg.vgg.mat - 1102
106237.jpg.vgg.mat - 1103
106240.jpg.vgg.mat - 1104
106247.jpg.vgg.mat - 1105
106257.jpg.vgg.mat - 1106
106270.jpg.vgg.mat - 1107
106286.jpg.vgg.mat - 1108
106289.jpg.vg

10793.jpg.vgg.mat - 1393
107931.jpg.vgg.mat - 1394
107936.jpg.vgg.mat - 1395
107938.jpg.vgg.mat - 1396
10794.jpg.vgg.mat - 1397
107955.jpg.vgg.mat - 1398
107956.jpg.vgg.mat - 1399
107957.jpg.vgg.mat - 1400
107967.jpg.vgg.mat - 1401
10797.jpg.vgg.mat - 1402
107970.jpg.vgg.mat - 1403
107972.jpg.vgg.mat - 1404
107978.jpg.vgg.mat - 1405
107979.jpg.vgg.mat - 1406
10798.jpg.vgg.mat - 1407
107983.jpg.vgg.mat - 1408
107992.jpg.vgg.mat - 1409
107999.jpg.vgg.mat - 1410
10801.jpg.vgg.mat - 1411
108012.jpg.vgg.mat - 1412
10802.jpg.vgg.mat - 1413
108022.jpg.vgg.mat - 1414
108023.jpg.vgg.mat - 1415
108026.jpg.vgg.mat - 1416
10803.jpg.vgg.mat - 1417
108030.jpg.vgg.mat - 1418
108031.jpg.vgg.mat - 1419
108049.jpg.vgg.mat - 1420
108061.jpg.vgg.mat - 1421
108064.jpg.vgg.mat - 1422
108067.jpg.vgg.mat - 1423
108070.jpg.vgg.mat - 1424
108085.jpg.vgg.mat - 1425
108091.jpg.vgg.mat - 1426
108094.jpg.vgg.mat - 1427
108097.jpg.vgg.mat - 1428
10810.jpg.vgg.mat - 1429
108106.jpg.vgg.mat - 1430
108112.jpg.vgg.mat -

...

142894.jpg.vgg.mat - 7492
142895.jpg.vgg.mat - 7493
142904.jpg.vgg.mat - 7494
14291.jpg.vgg.mat - 7495
142910.jpg.vgg.mat - 7496
142916.jpg.vgg.mat - 7497
142917.jpg.vgg.mat - 7498
142918.jpg.vgg.mat - 7499
14292.jpg.vgg.mat - 7500
142924.jpg.vgg.mat - 7501
14293.jpg.vgg.mat - 7502
142931.jpg.vgg.mat - 7503
142932.jpg.vgg.mat - 7504
142933.jpg.vgg.mat - 7505
14295.jpg.vgg.mat - 7506
142959.jpg.vgg.mat - 7507
142966.jpg.vgg.mat - 7508
142969.jpg.vgg.mat - 7509
142972.jpg.vgg.mat - 7510
142974.jpg.vgg.mat - 7511
14298.jpg.vgg.mat - 7512
142995.jpg.vgg.mat - 7513
143006.jpg.vgg.mat - 7514
143009.jpg.vgg.mat - 7515
143029.jpg.vgg.mat - 7516
143030.jpg.vgg.mat - 7517
143049.jpg.vgg.mat - 7518
143054.jpg.vgg.mat - 7519
143056.jpg.vgg.mat - 7520
143062.jpg.vgg.mat - 7521
143073.jpg.vgg.mat - 7522
143093.jpg.vgg.mat - 7523
143107.jpg.vgg.mat - 7524
143108.jpg.vgg.mat - 7525
143113.jpg.vgg.mat - 7526
143114.jpg.vgg.mat - 7527
143115.jpg.vgg.mat - 7528
143119.jpg.vgg.mat - 7529
143122.jpg.vgg.ma

144813.jpg.vgg.mat - 7812
144815.jpg.vgg.mat - 7813
144829.jpg.vgg.mat - 7814
144839.jpg.vgg.mat - 7815
14484.jpg.vgg.mat - 7816
144848.jpg.vgg.mat - 7817
14485.jpg.vgg.mat - 7818
144851.jpg.vgg.mat - 7819
144854.jpg.vgg.mat - 7820
144857.jpg.vgg.mat - 7821
144860.jpg.vgg.mat - 7822
144863.jpg.vgg.mat - 7823
144870.jpg.vgg.mat - 7824
144872.jpg.vgg.mat - 7825
144879.jpg.vgg.mat - 7826
14488.jpg.vgg.mat - 7827
144882.jpg.vgg.mat - 7828
144886.jpg.vgg.mat - 7829
144887.jpg.vgg.mat - 7830
144894.jpg.vgg.mat - 7831
144900.jpg.vgg.mat - 7832
144905.jpg.vgg.mat - 7833
144911.jpg.vgg.mat - 7834
144913.jpg.vgg.mat - 7835
144917.jpg.vgg.mat - 7836
14492.jpg.vgg.mat - 7837
144925.jpg.vgg.mat - 7838
144934.jpg.vgg.mat - 7839
144941.jpg.vgg.mat - 7840
144943.jpg.vgg.mat - 7841
144947.jpg.vgg.mat - 7842
144972.jpg.vgg.mat - 7843
144976.jpg.vgg.mat - 7844
144985.jpg.vgg.mat - 7845
144990.jpg.vgg.mat - 7846
144994.jpg.vgg.mat - 7847
144999.jpg.vgg.mat - 7848
145001.jpg.vgg.mat - 7849
145008.jpg.vgg.m

146593.jpg.vgg.mat - 8132
146596.jpg.vgg.mat - 8133
146598.jpg.vgg.mat - 8134
146600.jpg.vgg.mat - 8135
146609.jpg.vgg.mat - 8136
146614.jpg.vgg.mat - 8137
146619.jpg.vgg.mat - 8138
146620.jpg.vgg.mat - 8139
146622.jpg.vgg.mat - 8140
146624.jpg.vgg.mat - 8141
146628.jpg.vgg.mat - 8142
146645.jpg.vgg.mat - 8143
14665.jpg.vgg.mat - 8144
146652.jpg.vgg.mat - 8145
146656.jpg.vgg.mat - 8146
14666.jpg.vgg.mat - 8147
146660.jpg.vgg.mat - 8148
146664.jpg.vgg.mat - 8149
146666.jpg.vgg.mat - 8150
146667.jpg.vgg.mat - 8151
14667.jpg.vgg.mat - 8152
146682.jpg.vgg.mat - 8153
146685.jpg.vgg.mat - 8154
146688.jpg.vgg.mat - 8155
146701.jpg.vgg.mat - 8156
146706.jpg.vgg.mat - 8157
14671.jpg.vgg.mat - 8158
146717.jpg.vgg.mat - 8159
146723.jpg.vgg.mat - 8160
146724.jpg.vgg.mat - 8161
14673.jpg.vgg.mat - 8162
146732.jpg.vgg.mat - 8163
146737.jpg.vgg.mat - 8164
146738.jpg.vgg.mat - 8165
14674.jpg.vgg.mat - 8166
146744.jpg.vgg.mat - 8167
146747.jpg.vgg.mat - 8168
146761.jpg.vgg.mat - 8169
14677.jpg.vgg.mat 

148269.jpg.vgg.mat - 8449
148274.jpg.vgg.mat - 8450
148277.jpg.vgg.mat - 8451
148282.jpg.vgg.mat - 8452
148289.jpg.vgg.mat - 8453
148302.jpg.vgg.mat - 8454
148304.jpg.vgg.mat - 8455
148308.jpg.vgg.mat - 8456
148313.jpg.vgg.mat - 8457
148323.jpg.vgg.mat - 8458
148330.jpg.vgg.mat - 8459
148333.jpg.vgg.mat - 8460
148340.jpg.vgg.mat - 8461
148343.jpg.vgg.mat - 8462
148346.jpg.vgg.mat - 8463
148349.jpg.vgg.mat - 8464
148358.jpg.vgg.mat - 8465
148360.jpg.vgg.mat - 8466
148366.jpg.vgg.mat - 8467
148373.jpg.vgg.mat - 8468
148374.jpg.vgg.mat - 8469
148376.jpg.vgg.mat - 8470
148378.jpg.vgg.mat - 8471
148380.jpg.vgg.mat - 8472
148382.jpg.vgg.mat - 8473
148385.jpg.vgg.mat - 8474
148390.jpg.vgg.mat - 8475
148405.jpg.vgg.mat - 8476
148430.jpg.vgg.mat - 8477
148432.jpg.vgg.mat - 8478
148436.jpg.vgg.mat - 8479
14844.jpg.vgg.mat - 8480
148462.jpg.vgg.mat - 8481
148465.jpg.vgg.mat - 8482
148473.jpg.vgg.mat - 8483
148480.jpg.vgg.mat - 8484
148487.jpg.vgg.mat - 8485
148492.jpg.vgg.mat - 8486
14850.jpg.vgg

150088.jpg.vgg.mat - 8767
150090.jpg.vgg.mat - 8768
150093.jpg.vgg.mat - 8769
150096.jpg.vgg.mat - 8770
150111.jpg.vgg.mat - 8771
150113.jpg.vgg.mat - 8772
150116.jpg.vgg.mat - 8773
15012.jpg.vgg.mat - 8774
150125.jpg.vgg.mat - 8775
15013.jpg.vgg.mat - 8776
150134.jpg.vgg.mat - 8777
150140.jpg.vgg.mat - 8778
150144.jpg.vgg.mat - 8779
150145.jpg.vgg.mat - 8780
150149.jpg.vgg.mat - 8781
150150.jpg.vgg.mat - 8782
150158.jpg.vgg.mat - 8783
150166.jpg.vgg.mat - 8784
150168.jpg.vgg.mat - 8785
150169.jpg.vgg.mat - 8786
150175.jpg.vgg.mat - 8787
150176.jpg.vgg.mat - 8788
150182.jpg.vgg.mat - 8789
150194.jpg.vgg.mat - 8790
150202.jpg.vgg.mat - 8791
150212.jpg.vgg.mat - 8792
150215.jpg.vgg.mat - 8793
150217.jpg.vgg.mat - 8794
150228.jpg.vgg.mat - 8795
150231.jpg.vgg.mat - 8796
150232.jpg.vgg.mat - 8797
150245.jpg.vgg.mat - 8798
150249.jpg.vgg.mat - 8799
150250.jpg.vgg.mat - 8800
150257.jpg.vgg.mat - 8801
15026.jpg.vgg.mat - 8802
150265.jpg.vgg.mat - 8803
15027.jpg.vgg.mat - 8804
150277.jpg.vgg.m

In [31]:
np_data = sio.loadmat('gen_data/vgg_outputs_10k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_outputs_10k_label.vgg')
np_label = np_label['label']

classifier_2 = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, probability=True, random_state=random_state, 
                                        max_iter = 100))

print("Training start")
t1 = time.time()
classifier_2.fit(np_data, np_label)
print("Training done")
print("Training time for 50000 images took %0.3fs." % (time.time() - t1))


Training start


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarni

Training done
Training time for 50000 images took 3750.127s.


/home/pmadugundu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [37]:
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.svm import LinearSVC
"""
np_data = sio.loadmat('gen_data/vgg_outputs_10k_data.vgg')
np_data = np_data['data']
np_label = sio.loadmat('gen_data/vgg_outputs_10k_label.vgg')
np_label = np_label['label']
"""

print("Training start")
t1 = time.time()
classifier_3 = MultiOutputClassifier(LinearSVC(random_state=0, max_iter=100))
classifier_3.fit(np_data, np_label)

print("Training done")
print("Training time of MultiOutputClassifier for 50000 images took %0.3fs." % (time.time() - t1))

Training start
Training done
Training time of MultiOutputClassifier for 50000 images took 613.942s.


In [38]:
### testing the model
#inputpath = 'gen_data/vgg7avg/'
inputpath = 'gen_data/vgg_resize/'

test_list = file_list[50000:]
pred_list = []
label_list = []
i = 1
for file_name in test_list:
    print(file_name, '-', i)
    data = sio.loadmat(inputpath + file_name)
    data = data['features']
    #print(data.shape)
    f = file_name.split('.')
    label = id_to_label([f[0]])
    label = label.reshape((-1, 9))
    #print(label)
    #break
    pred = classifier_3.predict(data)
    i = i + 1
    label_list.append(label)
    pred_list.append(pred)

387326.jpg.vgg.mat - 1
387327.jpg.vgg.mat - 2
387361.jpg.vgg.mat - 3
387383.jpg.vgg.mat - 4
387393.jpg.vgg.mat - 5
387396.jpg.vgg.mat - 6
387397.jpg.vgg.mat - 7
387406.jpg.vgg.mat - 8
387419.jpg.vgg.mat - 9
387425.jpg.vgg.mat - 10
387435.jpg.vgg.mat - 11
38745.jpg.vgg.mat - 12
387455.jpg.vgg.mat - 13
387462.jpg.vgg.mat - 14
387465.jpg.vgg.mat - 15
387467.jpg.vgg.mat - 16
387468.jpg.vgg.mat - 17
387475.jpg.vgg.mat - 18
387483.jpg.vgg.mat - 19
387491.jpg.vgg.mat - 20
387502.jpg.vgg.mat - 21
387522.jpg.vgg.mat - 22
387525.jpg.vgg.mat - 23
387526.jpg.vgg.mat - 24
387533.jpg.vgg.mat - 25
387534.jpg.vgg.mat - 26
387545.jpg.vgg.mat - 27
387550.jpg.vgg.mat - 28
387555.jpg.vgg.mat - 29
387556.jpg.vgg.mat - 30
387564.jpg.vgg.mat - 31
387568.jpg.vgg.mat - 32
387576.jpg.vgg.mat - 33
387581.jpg.vgg.mat - 34
387594.jpg.vgg.mat - 35
387599.jpg.vgg.mat - 36
387603.jpg.vgg.mat - 37
387604.jpg.vgg.mat - 38
387608.jpg.vgg.mat - 39
387617.jpg.vgg.mat - 40
387624.jpg.vgg.mat - 41
387626.jpg.vgg.mat - 42
38

389971.jpg.vgg.mat - 430
389983.jpg.vgg.mat - 431
389986.jpg.vgg.mat - 432
389993.jpg.vgg.mat - 433
389995.jpg.vgg.mat - 434
389997.jpg.vgg.mat - 435
39.jpg.vgg.mat - 436
390019.jpg.vgg.mat - 437
390025.jpg.vgg.mat - 438
390029.jpg.vgg.mat - 439
39003.jpg.vgg.mat - 440
390034.jpg.vgg.mat - 441
390037.jpg.vgg.mat - 442
390041.jpg.vgg.mat - 443
390042.jpg.vgg.mat - 444
390057.jpg.vgg.mat - 445
390065.jpg.vgg.mat - 446
390066.jpg.vgg.mat - 447
39007.jpg.vgg.mat - 448
390091.jpg.vgg.mat - 449
390092.jpg.vgg.mat - 450
39010.jpg.vgg.mat - 451
390129.jpg.vgg.mat - 452
390133.jpg.vgg.mat - 453
390139.jpg.vgg.mat - 454
39014.jpg.vgg.mat - 455
390145.jpg.vgg.mat - 456
390147.jpg.vgg.mat - 457
390152.jpg.vgg.mat - 458
390155.jpg.vgg.mat - 459
390157.jpg.vgg.mat - 460
390160.jpg.vgg.mat - 461
390163.jpg.vgg.mat - 462
390165.jpg.vgg.mat - 463
390177.jpg.vgg.mat - 464
390178.jpg.vgg.mat - 465
390184.jpg.vgg.mat - 466
390188.jpg.vgg.mat - 467
39020.jpg.vgg.mat - 468
390201.jpg.vgg.mat - 469
390207.jp

392333.jpg.vgg.mat - 857
392347.jpg.vgg.mat - 858
392352.jpg.vgg.mat - 859
392360.jpg.vgg.mat - 860
392362.jpg.vgg.mat - 861
392380.jpg.vgg.mat - 862
392384.jpg.vgg.mat - 863
392386.jpg.vgg.mat - 864
392387.jpg.vgg.mat - 865
392395.jpg.vgg.mat - 866
392406.jpg.vgg.mat - 867
392408.jpg.vgg.mat - 868
392415.jpg.vgg.mat - 869
392416.jpg.vgg.mat - 870
392421.jpg.vgg.mat - 871
392430.jpg.vgg.mat - 872
392432.jpg.vgg.mat - 873
392436.jpg.vgg.mat - 874
392437.jpg.vgg.mat - 875
392439.jpg.vgg.mat - 876
392441.jpg.vgg.mat - 877
392456.jpg.vgg.mat - 878
392467.jpg.vgg.mat - 879
392479.jpg.vgg.mat - 880
392481.jpg.vgg.mat - 881
392482.jpg.vgg.mat - 882
392486.jpg.vgg.mat - 883
392503.jpg.vgg.mat - 884
392506.jpg.vgg.mat - 885
392514.jpg.vgg.mat - 886
392515.jpg.vgg.mat - 887
392523.jpg.vgg.mat - 888
392529.jpg.vgg.mat - 889
392536.jpg.vgg.mat - 890
392539.jpg.vgg.mat - 891
392546.jpg.vgg.mat - 892
392547.jpg.vgg.mat - 893
392559.jpg.vgg.mat - 894
392574.jpg.vgg.mat - 895
392577.jpg.vgg.mat - 896


39496.jpg.vgg.mat - 1281
394960.jpg.vgg.mat - 1282
394961.jpg.vgg.mat - 1283
394963.jpg.vgg.mat - 1284
394969.jpg.vgg.mat - 1285
394971.jpg.vgg.mat - 1286
394974.jpg.vgg.mat - 1287
39498.jpg.vgg.mat - 1288
394982.jpg.vgg.mat - 1289
394999.jpg.vgg.mat - 1290
395017.jpg.vgg.mat - 1291
39502.jpg.vgg.mat - 1292
395021.jpg.vgg.mat - 1293
395022.jpg.vgg.mat - 1294
395031.jpg.vgg.mat - 1295
395033.jpg.vgg.mat - 1296
395040.jpg.vgg.mat - 1297
395049.jpg.vgg.mat - 1298
395054.jpg.vgg.mat - 1299
395057.jpg.vgg.mat - 1300
395058.jpg.vgg.mat - 1301
395060.jpg.vgg.mat - 1302
395075.jpg.vgg.mat - 1303
39508.jpg.vgg.mat - 1304
395087.jpg.vgg.mat - 1305
395089.jpg.vgg.mat - 1306
395090.jpg.vgg.mat - 1307
39510.jpg.vgg.mat - 1308
395110.jpg.vgg.mat - 1309
395112.jpg.vgg.mat - 1310
395114.jpg.vgg.mat - 1311
395127.jpg.vgg.mat - 1312
395131.jpg.vgg.mat - 1313
395135.jpg.vgg.mat - 1314
395136.jpg.vgg.mat - 1315
39514.jpg.vgg.mat - 1316
395157.jpg.vgg.mat - 1317
395165.jpg.vgg.mat - 1318
395173.jpg.vgg.mat

397300.jpg.vgg.mat - 1710
397301.jpg.vgg.mat - 1711
397329.jpg.vgg.mat - 1712
397331.jpg.vgg.mat - 1713
397344.jpg.vgg.mat - 1714
397345.jpg.vgg.mat - 1715
397349.jpg.vgg.mat - 1716
397361.jpg.vgg.mat - 1717
397363.jpg.vgg.mat - 1718
39738.jpg.vgg.mat - 1719
397383.jpg.vgg.mat - 1720
397385.jpg.vgg.mat - 1721
397387.jpg.vgg.mat - 1722
397397.jpg.vgg.mat - 1723
397404.jpg.vgg.mat - 1724
397406.jpg.vgg.mat - 1725
397417.jpg.vgg.mat - 1726
397418.jpg.vgg.mat - 1727
397420.jpg.vgg.mat - 1728
397421.jpg.vgg.mat - 1729
397422.jpg.vgg.mat - 1730
397427.jpg.vgg.mat - 1731
397430.jpg.vgg.mat - 1732
397443.jpg.vgg.mat - 1733
397458.jpg.vgg.mat - 1734
397466.jpg.vgg.mat - 1735
397470.jpg.vgg.mat - 1736
397482.jpg.vgg.mat - 1737
397483.jpg.vgg.mat - 1738
397492.jpg.vgg.mat - 1739
397531.jpg.vgg.mat - 1740
397548.jpg.vgg.mat - 1741
397554.jpg.vgg.mat - 1742
39756.jpg.vgg.mat - 1743
397586.jpg.vgg.mat - 1744
397588.jpg.vgg.mat - 1745
39760.jpg.vgg.mat - 1746
397601.jpg.vgg.mat - 1747
397604.jpg.vgg.

...

87973.jpg.vgg.mat - 22094
87976.jpg.vgg.mat - 22095
87986.jpg.vgg.mat - 22096
87987.jpg.vgg.mat - 22097
880.jpg.vgg.mat - 22098
88002.jpg.vgg.mat - 22099
88010.jpg.vgg.mat - 22100
88016.jpg.vgg.mat - 22101
8802.jpg.vgg.mat - 22102
88044.jpg.vgg.mat - 22103
88047.jpg.vgg.mat - 22104
88051.jpg.vgg.mat - 22105
88053.jpg.vgg.mat - 22106
88071.jpg.vgg.mat - 22107
88075.jpg.vgg.mat - 22108
88089.jpg.vgg.mat - 22109
88091.jpg.vgg.mat - 22110
88093.jpg.vgg.mat - 22111
88096.jpg.vgg.mat - 22112
8810.jpg.vgg.mat - 22113
88109.jpg.vgg.mat - 22114
88117.jpg.vgg.mat - 22115
88125.jpg.vgg.mat - 22116
88129.jpg.vgg.mat - 22117
88133.jpg.vgg.mat - 22118
88137.jpg.vgg.mat - 22119
88138.jpg.vgg.mat - 22120
88146.jpg.vgg.mat - 22121
8815.jpg.vgg.mat - 22122
88154.jpg.vgg.mat - 22123
88157.jpg.vgg.mat - 22124
88158.jpg.vgg.mat - 22125
88168.jpg.vgg.mat - 22126
8817.jpg.vgg.mat - 22127
88172.jpg.vgg.mat - 22128
88175.jpg.vgg.mat - 22129
88179.jpg.vgg.mat - 22130
88190.jpg.vgg.mat - 22131
88191.jpg.vgg.mat 

90489.jpg.vgg.mat - 22522
90490.jpg.vgg.mat - 22523
90492.jpg.vgg.mat - 22524
905.jpg.vgg.mat - 22525
90509.jpg.vgg.mat - 22526
90513.jpg.vgg.mat - 22527
9052.jpg.vgg.mat - 22528
90538.jpg.vgg.mat - 22529
90542.jpg.vgg.mat - 22530
90548.jpg.vgg.mat - 22531
90550.jpg.vgg.mat - 22532
90552.jpg.vgg.mat - 22533
90558.jpg.vgg.mat - 22534
90560.jpg.vgg.mat - 22535
90568.jpg.vgg.mat - 22536
90573.jpg.vgg.mat - 22537
90574.jpg.vgg.mat - 22538
90577.jpg.vgg.mat - 22539
90578.jpg.vgg.mat - 22540
9058.jpg.vgg.mat - 22541
90581.jpg.vgg.mat - 22542
90584.jpg.vgg.mat - 22543
90587.jpg.vgg.mat - 22544
90588.jpg.vgg.mat - 22545
90589.jpg.vgg.mat - 22546
90593.jpg.vgg.mat - 22547
90595.jpg.vgg.mat - 22548
906.jpg.vgg.mat - 22549
9060.jpg.vgg.mat - 22550
90603.jpg.vgg.mat - 22551
90609.jpg.vgg.mat - 22552
90610.jpg.vgg.mat - 22553
90618.jpg.vgg.mat - 22554
90623.jpg.vgg.mat - 22555
90633.jpg.vgg.mat - 22556
9064.jpg.vgg.mat - 22557
90640.jpg.vgg.mat - 22558
90649.jpg.vgg.mat - 22559
90650.jpg.vgg.mat - 

92966.jpg.vgg.mat - 22948
92970.jpg.vgg.mat - 22949
92974.jpg.vgg.mat - 22950
92992.jpg.vgg.mat - 22951
930.jpg.vgg.mat - 22952
93005.jpg.vgg.mat - 22953
93012.jpg.vgg.mat - 22954
93019.jpg.vgg.mat - 22955
93023.jpg.vgg.mat - 22956
93028.jpg.vgg.mat - 22957
93032.jpg.vgg.mat - 22958
93055.jpg.vgg.mat - 22959
93061.jpg.vgg.mat - 22960
93062.jpg.vgg.mat - 22961
93065.jpg.vgg.mat - 22962
93066.jpg.vgg.mat - 22963
93084.jpg.vgg.mat - 22964
93091.jpg.vgg.mat - 22965
93097.jpg.vgg.mat - 22966
93099.jpg.vgg.mat - 22967
931.jpg.vgg.mat - 22968
93100.jpg.vgg.mat - 22969
93113.jpg.vgg.mat - 22970
93126.jpg.vgg.mat - 22971
9313.jpg.vgg.mat - 22972
9314.jpg.vgg.mat - 22973
9315.jpg.vgg.mat - 22974
93150.jpg.vgg.mat - 22975
93155.jpg.vgg.mat - 22976
93156.jpg.vgg.mat - 22977
93164.jpg.vgg.mat - 22978
93165.jpg.vgg.mat - 22979
93166.jpg.vgg.mat - 22980
93189.jpg.vgg.mat - 22981
93192.jpg.vgg.mat - 22982
93197.jpg.vgg.mat - 22983
93205.jpg.vgg.mat - 22984
9321.jpg.vgg.mat - 22985
93210.jpg.vgg.mat - 

95524.jpg.vgg.mat - 23370
95528.jpg.vgg.mat - 23371
9553.jpg.vgg.mat - 23372
95531.jpg.vgg.mat - 23373
95545.jpg.vgg.mat - 23374
9555.jpg.vgg.mat - 23375
95555.jpg.vgg.mat - 23376
9556.jpg.vgg.mat - 23377
95561.jpg.vgg.mat - 23378
95566.jpg.vgg.mat - 23379
95572.jpg.vgg.mat - 23380
9558.jpg.vgg.mat - 23381
95584.jpg.vgg.mat - 23382
95599.jpg.vgg.mat - 23383
95605.jpg.vgg.mat - 23384
95610.jpg.vgg.mat - 23385
95611.jpg.vgg.mat - 23386
95620.jpg.vgg.mat - 23387
95622.jpg.vgg.mat - 23388
9563.jpg.vgg.mat - 23389
95630.jpg.vgg.mat - 23390
95631.jpg.vgg.mat - 23391
95635.jpg.vgg.mat - 23392
95636.jpg.vgg.mat - 23393
9564.jpg.vgg.mat - 23394
95641.jpg.vgg.mat - 23395
95642.jpg.vgg.mat - 23396
95654.jpg.vgg.mat - 23397
95656.jpg.vgg.mat - 23398
95659.jpg.vgg.mat - 23399
9566.jpg.vgg.mat - 23400
95664.jpg.vgg.mat - 23401
95667.jpg.vgg.mat - 23402
95683.jpg.vgg.mat - 23403
95687.jpg.vgg.mat - 23404
95693.jpg.vgg.mat - 23405
95699.jpg.vgg.mat - 23406
95701.jpg.vgg.mat - 23407
95704.jpg.vgg.mat -

98165.jpg.vgg.mat - 23798
9817.jpg.vgg.mat - 23799
98175.jpg.vgg.mat - 23800
98179.jpg.vgg.mat - 23801
98197.jpg.vgg.mat - 23802
98206.jpg.vgg.mat - 23803
98208.jpg.vgg.mat - 23804
98211.jpg.vgg.mat - 23805
98217.jpg.vgg.mat - 23806
98219.jpg.vgg.mat - 23807
98223.jpg.vgg.mat - 23808
98226.jpg.vgg.mat - 23809
98236.jpg.vgg.mat - 23810
98245.jpg.vgg.mat - 23811
98248.jpg.vgg.mat - 23812
98249.jpg.vgg.mat - 23813
98252.jpg.vgg.mat - 23814
9826.jpg.vgg.mat - 23815
98260.jpg.vgg.mat - 23816
98268.jpg.vgg.mat - 23817
98272.jpg.vgg.mat - 23818
98278.jpg.vgg.mat - 23819
98281.jpg.vgg.mat - 23820
98291.jpg.vgg.mat - 23821
98293.jpg.vgg.mat - 23822
98302.jpg.vgg.mat - 23823
98303.jpg.vgg.mat - 23824
98304.jpg.vgg.mat - 23825
98307.jpg.vgg.mat - 23826
98309.jpg.vgg.mat - 23827
9831.jpg.vgg.mat - 23828
98314.jpg.vgg.mat - 23829
98320.jpg.vgg.mat - 23830
98325.jpg.vgg.mat - 23831
98330.jpg.vgg.mat - 23832
98338.jpg.vgg.mat - 23833
98340.jpg.vgg.mat - 23834
98342.jpg.vgg.mat - 23835
98345.jpg.vgg.m

In [39]:
def verify_correctness(pred1, test_labels, threshold=0):
    cnt=0
    i = 1
    for x, y in zip(pred1, test_labels):
        #x1= np.array(x)
        #y1 = np.array(y)
        #print(x, y)
        #break
        c=0
        for a, b in zip(x[0], y[0]):
            #print(a, b)
            #break
            if(a!=b): c+=1
        if c<=threshold:
            cnt+=1
        #if i > 50 and i < 100:
            #print(i,'-',cnt,'-', x1, y1)
        i = i + 1
    accuracy = cnt/len(test_labels)
    return accuracy

#for p in pred_list[:50]:
#    print(p)
#print(label_list[0].shape)
#print(pred_list[1].shape, label_list[1].shape)
#pred_list = np.asarray(pred_list).reshape((-1, 9))
#label_list = np.asarray(label_list).reshape(-1, 9)

#print(pred_list.shape, label_list.shape)

print("Accuracy score for strict match is ", verify_correctness(pred_list, label_list, 0)*100)
print("Accuracy score for maximum of 1 mistmatch is ", verify_correctness(pred_list, label_list, 1)*100)
print("Accuracy score for maximum of 2 mistmatch is ", verify_correctness(pred_list, label_list, 2)*100)
print("Accuracy score for maximum of 3 mistmatch is ", verify_correctness(pred_list, label_list, 3)*100)
print("Accuracy score for maximum of 4 mistmatch is ", verify_correctness(pred_list, label_list, 4)*100)
print("Accuracy score for maximum of 5 mistmatch is ", verify_correctness(pred_list, label_list, 5)*100)
print("Accuracy score for maximum of 6 mistmatch is ", verify_correctness(pred_list, label_list, 6)*100)
print("Accuracy score for maximum of 7 mistmatch is ", verify_correctness(pred_list, label_list, 7)*100)
print("Accuracy score for maximum of 8 mistmatch is ", verify_correctness(pred_list, label_list, 8)*100)
print("Accuracy score for maximum of 9 mistmatch is ", verify_correctness(pred_list, label_list, 9)*100)

    

Accuracy score for strict match is  0.33142762449250146
Accuracy score for maximum of 1 mistmatch is  3.123705360841826
Accuracy score for maximum of 2 mistmatch is  13.323390504598558
Accuracy score for maximum of 3 mistmatch is  39.66360096114011
Accuracy score for maximum of 4 mistmatch is  72.77736349324717
Accuracy score for maximum of 5 mistmatch is  94.15030242770735
Accuracy score for maximum of 6 mistmatch is  99.01400281713481
Accuracy score for maximum of 7 mistmatch is  99.94614301101997
Accuracy score for maximum of 8 mistmatch is  100.0
Accuracy score for maximum of 9 mistmatch is  100.0
